In [1]:
# Journey North Peak Migration

In [2]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-19-2022

# geoJSONifier

In [3]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Load file

In [4]:
JNorth1996_2020_df = pd.read_csv("UWMadisonArb_JNorth_Project_MonarchsAndMilkweed_1996-2020.csv")
JNorth1996_2020_df.head()

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


id sighting_date                species  number  latitude  \
0  850841703    1996-12-16  Monarch Adult Sighted       1    27.594   
1  851824801    1996-12-06  Monarch Adult Sighted       1    30.420   
2  854409931    1996-09-21  Monarch Adult Sighted       1    40.817   
3  854557206    1996-09-21  Monarch Adult Sighted       1    40.817   
4  855061862    1997-01-01  Monarch Adult Sighted       1    34.244   

   longitude  flag_location  \
0    -81.362              0   
1    -87.224              0   
2    -96.710              0   
3    -96.710              0   
4    -77.946              0   

                                            comments  \
0  On the ballfield a solitary monarch butterfly ...   
1  Our 5th grade Discovery students who went alon...   
2  We saw several hundred monarchs in our school ...   
3  Over a hundered monarchs in our school butterf...   
4  I Thought I Saw It, But I Was Suprised To See ...   

                           school  grade    image_url  customerid  number_old  \
0  Walker memorial junior academy   10.0  -999999.999         135 -999999.999   
1      Hambrick elementary school    5.0  -999999.999         150 -999999.999   
2                     -999999.999    5.0  -999999.999     9701054 -999999.999   
3     Fredstrom elementary school    6.0  -999999.999         175 -999999.999   
4                         Gregory    3.0  -999999.999         181 -999999.999   

   flag_number        species_old  flag_species  
0            1  Monarch Butterfly             1  
1            1  Monarch Butterfly             1  
2            1  Monarch Butterfly             1  
3            1  Monarch Butterfly             1  
4            1  Monarch Butterfly             1

# Data Exploratory

In [5]:
JNorth1996_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227433 entries, 0 to 227432
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             227433 non-null  int64  
 1   sighting_date  227433 non-null  object 
 2   species        227433 non-null  object 
 3   number         227433 non-null  int64  
 4   latitude       227433 non-null  float64
 5   longitude      227433 non-null  float64
 6   flag_location  227433 non-null  int64  
 7   comments       227433 non-null  object 
 8   school         227433 non-null  object 
 9   grade          227433 non-null  float64
 10  image_url      227433 non-null  object 
 11  customerid     227433 non-null  int64  
 12  number_old     227433 non-null  float64
 13  flag_number    227433 non-null  int64  
 14  species_old    227433 non-null  object 
 15  flag_species   227433 non-null  int64  
dtypes: float64(4), int64(6), object(6)
memory usage: 27.8+ MB


In [6]:
str(JNorth1996_2020_df.columns.tolist())

"['id', 'sighting_date', 'species', 'number', 'latitude', 'longitude', 'flag_location', 'comments', 'school', 'grade', 'image_url', 'customerid', 'number_old', 'flag_number', 'species_old', 'flag_species']"

In [7]:
JNorth1996_2020_df.describe()

id        number       latitude      longitude  \
count  2.274330e+05  2.274330e+05  227433.000000  227433.000000   
mean   1.420650e+09  1.186529e+02      38.400690     -87.569965   
std    1.456887e+08  2.209080e+04       5.268651      10.880362   
min    8.508417e+08 -1.700000e+01     -12.100000    -165.107000   
25%    1.377751e+09  1.000000e+00      34.723000     -95.268000   
50%    1.477448e+09  1.000000e+00      39.503000     -86.787000   
75%    1.526383e+09  3.000000e+00      42.409000     -79.255000   
max    1.526525e+09  1.000000e+07      65.136000     -52.883000   

       flag_location          grade    customerid    number_old  \
count  227433.000000  227433.000000  2.274330e+05  2.274330e+05   
mean        0.598018 -907386.217597  9.451737e+06 -2.844270e+05   
std         0.490299  289894.342744  2.193961e+06  4.518136e+05   
min         0.000000 -999999.999000  0.000000e+00 -1.000000e+06   
25%         0.000000 -999999.999000  9.941050e+06 -1.000000e+06   
50%         1.000000 -999999.999000  9.977663e+06  1.000000e+00   
75%         1.000000 -999999.999000  1.000029e+07  3.000000e+00   
max         1.000000    1112.000000  1.004256e+07  1.000000e+07   

         flag_number   flag_species  
count  227433.000000  227433.000000  
mean        0.294816       0.130324  
std         0.467089       0.336660  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         1.000000       0.000000  
max         2.000000       1.000000

In [8]:
JNorth1996_2020_df.columns.tolist()

['id',
 'sighting_date',
 'species',
 'number',
 'latitude',
 'longitude',
 'flag_location',
 'comments',
 'school',
 'grade',
 'image_url',
 'customerid',
 'number_old',
 'flag_number',
 'species_old',
 'flag_species']

In [9]:
JNorth1996_2020_df.shape

(227433, 16)

In [10]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].unique())

[ 850841703  851824801  854409931 ... 1526524404 1526524405 1526525021]
['1996-12-16' '1996-12-06' '1996-09-21' ... '2020-12-29' '2020-12-30'
 '2020-12-31']
['Monarch Adult Sighted' 'Monarch Adult (FIRST sighted)'
 'Monarch Egg (FIRST sighted)' 'Monarch (OTHER Observations)'
 'Monarch Larva (FIRST sighted)' 'Monarch PEAK Migration'
 'Milkweed (FIRST sighted)' 'Monarch Fall Roost' 'Monarch Egg Sighted'
 'Monarch Larva Sighted' 'Monarch, Captive-Reared' 'Milkweed Sighted']
[       1      150       20       89       30     2000       50     7500
       23      200        5      300        4      100       71      120
      325       21       28     5000    20000       36       25        6
       54     1000      160       24       60       37        2     3000
  1000000       10  2000000       40       85       65       15        8
      500       14      112       75       80    15000       18    10000
      912   100000      400      800       12       35      600       16
    25000    

In [11]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].value_counts())

850841703     1
1526350827    1
1526350816    1
1526350817    1
1526350818    1
             ..
1432512422    1
1432519704    1
1432562236    1
1432562641    1
1526525021    1
Name: id, Length: 227433, dtype: int64
2018-08-22    392
2018-10-03    377
2018-09-06    353
2019-09-19    337
2018-09-12    335
             ... 
2015-12-31      1
2015-11-24      1
2015-11-22      1
2006-07-30      1
1996-12-16      1
Name: sighting_date, Length: 7491, dtype: int64
Monarch Adult Sighted            102306
Monarch Adult (FIRST sighted)     36409
Monarch Larva Sighted             17378
Monarch, Captive-Reared           14339
Monarch (OTHER Observations)      13581
Monarch Egg Sighted                8736
Milkweed (FIRST sighted)           8465
Monarch PEAK Migration             7689
Monarch Egg (FIRST sighted)        6876
Monarch Fall Roost                 5035
Monarch Larva (FIRST sighted)      4898
Milkweed Sighted                   1721
Name: species, dtype: int64
1       139319
2        24169
3

In [12]:
# Find null values
for column in JNorth1996_2020_df.columns:
   print(f"Column {column} has {JNorth1996_2020_df[column].isnull().sum()} null values")

Column id has 0 null values
Column sighting_date has 0 null values
Column species has 0 null values
Column number has 0 null values
Column latitude has 0 null values
Column longitude has 0 null values
Column flag_location has 0 null values
Column comments has 0 null values
Column school has 0 null values
Column grade has 0 null values
Column image_url has 0 null values
Column customerid has 0 null values
Column number_old has 0 null values
Column flag_number has 0 null values
Column species_old has 0 null values
Column flag_species has 0 null values


In [13]:
# Find duplicate entries
print(f"Duplicate entries: {JNorth1996_2020_df.duplicated().sum()}")

Duplicate entries: 0


In [14]:
# filtering by type of observation (Monarch Peak Migration)
JNorth_Peak_Migration_df = JNorth1996_2020_df[JNorth1996_2020_df['species']=='Monarch PEAK Migration']
JNorth_Peak_Migration_df.head()

id sighting_date                 species  number  latitude  \
3541  966119477    2000-08-08  Monarch PEAK Migration       1    47.066   
3544  966258999    2000-08-15  Monarch PEAK Migration       1    44.957   
3547  966374597    2000-08-15  Monarch PEAK Migration       1    45.009   
3551  966612408    2000-08-17  Monarch PEAK Migration       1    45.050   
3572  966879879    2000-08-20  Monarch PEAK Migration       1    46.926   

      longitude  flag_location  \
3541    -94.504              0   
3544    -93.540              0   
3547    -93.340              0   
3551    -93.210              0   
3572    -96.850              0   

                                               comments        school  \
3541  The first signs of the migration in Minnesota ...   -999999.999   
3544  I observed the first big pulse of fall migrati...   -999999.999   
3547  Lots of Monarch activity today!  We counted 58...  Breck school   
3551  The day was an exciting one. We had about 75 b...   -999999.999   
3572  It was totally awesome. It was about 4:30pm on...   -999999.999   

           grade    image_url  customerid  number_old  flag_number  \
3541       0.000  -999999.999     9710973       0.000            2   
3544 -999999.999  -999999.999     9937188 -999999.999            1   
3547       4.000  -999999.999     9704118       0.000            2   
3551       0.000  -999999.999        2266       0.000            2   
3572 -999999.999  -999999.999     9900260 -999999.999            1   

                 species_old  flag_species  
3541  Monarch PEAK Migration             0  
3544  Monarch PEAK Migration             0  
3547  Monarch PEAK Migration             0  
3551  Monarch PEAK Migration             0  
3572  Monarch PEAK Migration             0

In [15]:
JNorth_Peak_Migration_df.to_csv('JNorth_Peak_Migration.csv', header=True, mode='w' )

In [16]:
JNorth_Peak_Migration_df.shape

(7689, 16)

# Format file for geoJSON 

In [17]:
JNorth_Peak_Migration_df = pd.DataFrame(JNorth_Peak_Migration_df)
JNorth_Peak_Migration_df.drop(['id','flag_location', 'comments','school','grade','image_url','customerid','number_old','flag_number','species_old','flag_species'], axis=1, inplace=True)
JNorth_Peak_Migration_df.head()

sighting_date                 species  number  latitude  longitude
3541    2000-08-08  Monarch PEAK Migration       1    47.066    -94.504
3544    2000-08-15  Monarch PEAK Migration       1    44.957    -93.540
3547    2000-08-15  Monarch PEAK Migration       1    45.009    -93.340
3551    2000-08-17  Monarch PEAK Migration       1    45.050    -93.210
3572    2000-08-20  Monarch PEAK Migration       1    46.926    -96.850

In [18]:
cols = ['sighting_date', 'species', 'number', 'latitude', 'longitude']

In [19]:
JNorth_Peak_Migration_df =JNorth_Peak_Migration_df.groupby(['sighting_date', 'species', 'latitude', 'longitude']).sum().reset_index()
JNorth_Peak_Migration_df.head()

sighting_date                 species  latitude  longitude  number
0    2000-08-08  Monarch PEAK Migration    47.066    -94.504       1
1    2000-08-15  Monarch PEAK Migration    44.957    -93.540       1
2    2000-08-15  Monarch PEAK Migration    45.009    -93.340       1
3    2000-08-16  Monarch PEAK Migration    45.352    -94.313       1
4    2000-08-16  Monarch PEAK Migration    46.001    -94.341       1

In [20]:
JNorth_Peak_Migration_df.columns.tolist()

['sighting_date', 'species', 'latitude', 'longitude', 'number']

In [21]:
JNorth_Peak_Migration_df.head()

sighting_date                 species  latitude  longitude  number
0    2000-08-08  Monarch PEAK Migration    47.066    -94.504       1
1    2000-08-15  Monarch PEAK Migration    44.957    -93.540       1
2    2000-08-15  Monarch PEAK Migration    45.009    -93.340       1
3    2000-08-16  Monarch PEAK Migration    45.352    -94.313       1
4    2000-08-16  Monarch PEAK Migration    46.001    -94.341       1

In [22]:
JNorth_Peak_Migration_df.shape

(7366, 5)

In [23]:
JNorth_Peak_Migration_year_df = JNorth_Peak_Migration_df
JNorth_Peak_Migration_year_df['year'] = pd.DatetimeIndex(JNorth_Peak_Migration_df['sighting_date']).year

# 2001

In [24]:
JNorth_Peak_Migration_2001_df = JNorth_Peak_Migration_year_df[JNorth_Peak_Migration_year_df['year']==2001]
JNorth_Peak_Migration_2001_df.tail()

sighting_date                 species  latitude  longitude  number  year
418    2001-10-23  Monarch PEAK Migration    30.610    -96.386       1  2001
419    2001-10-24  Monarch PEAK Migration    31.688    -89.111       1  2001
420    2001-10-29  Monarch PEAK Migration    25.670   -100.320       1  2001
421    2001-11-01  Monarch PEAK Migration    29.984    -90.166       1  2001
422    2001-11-16  Monarch PEAK Migration    29.984    -90.166       1  2001

In [25]:
JNorth_Peak_Migration_2001_df.shape

(250, 6)

In [26]:
JNorth_Peak_Migration_2001_df = pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_df.drop(['year'], axis=1, inplace=True)

In [27]:
JNorth_Peak_Migration_2001_df['month'] = pd.DatetimeIndex(JNorth_Peak_Migration_2001_df['sighting_date']).month

## January

In [28]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_1_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==1]
JNorth_Peak_Migration_2001_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [29]:
JNorth_Peak_Migration_2001_1_df.shape

(0, 6)

In [30]:
JNorth_Peak_Migration_2001_1_df = pd.DataFrame(JNorth_Peak_Migration_2001_1_df)
JNorth_Peak_Migration_2001_1_df.drop(['month'], axis=1, inplace=True)

In [31]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_1_df['latitude'] = JNorth_Peak_Migration_2001_1_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_1_df['longitude'] = JNorth_Peak_Migration_2001_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [32]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_1_df, cols)

In [33]:
file = open("JNorth_Peak_Migration_2001_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [34]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_2_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==2]
JNorth_Peak_Migration_2001_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [35]:
JNorth_Peak_Migration_2001_2_df.shape

(0, 6)

In [36]:
JNorth_Peak_Migration_2001_2_df= pd.DataFrame(JNorth_Peak_Migration_2001_2_df)
JNorth_Peak_Migration_2001_2_df.drop(['month'], axis=1, inplace=True)

In [37]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_2_df['latitude'] = JNorth_Peak_Migration_2001_2_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_2_df['longitude'] = JNorth_Peak_Migration_2001_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [38]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_2_df, cols)

In [39]:
file = open("JNorth_Peak_Migration_2001_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [40]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_3_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==3]
JNorth_Peak_Migration_2001_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [41]:
JNorth_Peak_Migration_2001_3_df.shape

(0, 6)

In [42]:
JNorth_Peak_Migration_2001_3_df =pd.DataFrame(JNorth_Peak_Migration_2001_3_df)
JNorth_Peak_Migration_2001_3_df.drop(['month'], axis=1, inplace=True)

In [43]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_3_df['latitude'] = JNorth_Peak_Migration_2001_3_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_3_df['longitude'] = JNorth_Peak_Migration_2001_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [44]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_3_df, cols)

In [45]:
file = open("JNorth_Peak_Migration_2001_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [46]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_4_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==4]
JNorth_Peak_Migration_2001_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [47]:
JNorth_Peak_Migration_2001_4_df.shape

(0, 6)

In [48]:
JNorth_Peak_Migration_2001_4_df = pd.DataFrame(JNorth_Peak_Migration_2001_4_df)
JNorth_Peak_Migration_2001_4_df.drop(['month'], axis=1, inplace=True)

In [49]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_4_df['latitude'] = JNorth_Peak_Migration_2001_4_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_4_df['longitude'] = JNorth_Peak_Migration_2001_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [50]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_4_df, cols)

In [51]:
file = open("JNorth_Peak_Migration_2001_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [52]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_5_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==5]
JNorth_Peak_Migration_2001_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [53]:
JNorth_Peak_Migration_2001_5_df.shape

(0, 6)

In [54]:
JNorth_Peak_Migration_2001_5_df =pd.DataFrame(JNorth_Peak_Migration_2001_5_df)
JNorth_Peak_Migration_2001_5_df.drop(['month'], axis=1, inplace=True)

In [55]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_5_df['latitude'] = JNorth_Peak_Migration_2001_5_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_5_df['longitude'] = JNorth_Peak_Migration_2001_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [56]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_5_df, cols)

In [57]:
file = open("JNorth_Peak_Migration_2001_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [58]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_6_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==6]
JNorth_Peak_Migration_2001_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [59]:
JNorth_Peak_Migration_2001_6_df.shape

(0, 6)

In [60]:
JNorth_Peak_Migration_2001_6_df=pd.DataFrame(JNorth_Peak_Migration_2001_6_df)
JNorth_Peak_Migration_2001_6_df.drop(['month'], axis=1, inplace=True)

In [61]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_6_df['latitude'] = JNorth_Peak_Migration_2001_6_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_6_df['longitude'] = JNorth_Peak_Migration_2001_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [62]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_6_df, cols)

In [63]:
file = open("JNorth_Peak_Migration_2001_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [64]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_7_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==7]
JNorth_Peak_Migration_2001_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [65]:
JNorth_Peak_Migration_2001_7_df.shape

(0, 6)

In [66]:
JNorth_Peak_Migration_2001_7_df = pd.DataFrame(JNorth_Peak_Migration_2001_7_df)
JNorth_Peak_Migration_2001_7_df.drop(['month'], axis=1, inplace=True)

In [67]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_7_df['latitude'] = JNorth_Peak_Migration_2001_7_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_7_df['longitude'] = JNorth_Peak_Migration_2001_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [68]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_7_df, cols)

In [69]:
file = open("JNorth_Peak_Migration_2001_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [70]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_8_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==8]
JNorth_Peak_Migration_2001_8_df.tail()

sighting_date                 species  latitude  longitude  number  month
191    2001-08-28  Monarch PEAK Migration    41.908    -93.606       1      8
192    2001-08-28  Monarch PEAK Migration    42.953    -93.737       1      8
193    2001-08-28  Monarch PEAK Migration    47.368    -97.410       1      8
194    2001-08-29  Monarch PEAK Migration    42.650    -80.820       1      8
195    2001-08-30  Monarch PEAK Migration    43.363    -92.168       1      8

In [71]:
JNorth_Peak_Migration_2001_8_df.shape

(23, 6)

In [72]:
JNorth_Peak_Migration_2001_8_df = pd.DataFrame(JNorth_Peak_Migration_2001_8_df)
JNorth_Peak_Migration_2001_8_df.drop(['month'], axis=1, inplace=True)

In [73]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_8_df['latitude'] = JNorth_Peak_Migration_2001_8_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_8_df['longitude'] = JNorth_Peak_Migration_2001_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [74]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_8_df, cols)

In [75]:
file = open("JNorth_Peak_Migration_2001_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [76]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_9_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==9]
JNorth_Peak_Migration_2001_9_df.tail()

sighting_date                 species  latitude  longitude  number  month
339    2001-09-30  Monarch PEAK Migration    29.757    -95.365       1      9
340    2001-09-30  Monarch PEAK Migration    32.756    -99.380       1      9
341    2001-09-30  Monarch PEAK Migration    33.995    -83.350       1      9
342    2001-09-30  Monarch PEAK Migration    35.011   -120.459       1      9
343    2001-09-30  Monarch PEAK Migration    35.422   -100.839       1      9

In [77]:
JNorth_Peak_Migration_2001_9_df.shape

(148, 6)

In [78]:
JNorth_Peak_Migration_2001_9_df = pd.DataFrame(JNorth_Peak_Migration_2001_9_df)
JNorth_Peak_Migration_2001_9_df.drop(['month'], axis=1, inplace=True)

In [79]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_9_df['latitude'] = JNorth_Peak_Migration_2001_9_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_9_df['longitude'] = JNorth_Peak_Migration_2001_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [80]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_9_df, cols)

In [81]:
file = open("JNorth_Peak_Migration_2001_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [82]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_10_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==10]
JNorth_Peak_Migration_2001_10_df.tail()

sighting_date                 species  latitude  longitude  number  month
416    2001-10-21  Monarch PEAK Migration    20.350   -100.240       1     10
417    2001-10-22  Monarch PEAK Migration    31.688    -89.111       1     10
418    2001-10-23  Monarch PEAK Migration    30.610    -96.386       1     10
419    2001-10-24  Monarch PEAK Migration    31.688    -89.111       1     10
420    2001-10-29  Monarch PEAK Migration    25.670   -100.320       1     10

In [83]:
JNorth_Peak_Migration_2001_10_df.shape

(77, 6)

In [84]:
JNorth_Peak_Migration_2001_10_df = pd.DataFrame(JNorth_Peak_Migration_2001_10_df)
JNorth_Peak_Migration_2001_10_df.drop(['month'], axis=1, inplace=True)

In [85]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_10_df['latitude'] = JNorth_Peak_Migration_2001_10_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_10_df['longitude'] = JNorth_Peak_Migration_2001_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [86]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_10_df, cols)

In [87]:
file = open("JNorth_Peak_Migration_2001_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [88]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_11_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==11]
JNorth_Peak_Migration_2001_11_df.tail()

sighting_date                 species  latitude  longitude  number  month
421    2001-11-01  Monarch PEAK Migration    29.984    -90.166       1     11
422    2001-11-16  Monarch PEAK Migration    29.984    -90.166       1     11

In [89]:
JNorth_Peak_Migration_2001_11_df.shape

(2, 6)

In [90]:
JNorth_Peak_Migration_2001_11_df = pd.DataFrame(JNorth_Peak_Migration_2001_11_df)
JNorth_Peak_Migration_2001_11_df.drop(['month'], axis=1, inplace=True)

In [91]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_11_df['latitude'] = JNorth_Peak_Migration_2001_11_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_11_df['longitude'] = JNorth_Peak_Migration_2001_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [92]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_11_df, cols)

In [93]:
file = open("JNorth_Peak_Migration_2001_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [94]:
JNorth_Peak_Migration_2001_month_df =pd.DataFrame(JNorth_Peak_Migration_2001_df)
JNorth_Peak_Migration_2001_12_df = JNorth_Peak_Migration_2001_month_df[JNorth_Peak_Migration_2001_month_df['month']==12]
JNorth_Peak_Migration_2001_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [95]:
JNorth_Peak_Migration_2001_12_df.shape

(0, 6)

In [96]:
JNorth_Peak_Migration_2001_12_df = pd.DataFrame(JNorth_Peak_Migration_2001_12_df)
JNorth_Peak_Migration_2001_12_df.drop(['month'], axis=1, inplace=True)

In [97]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2001_12_df['latitude'] = JNorth_Peak_Migration_2001_12_df['latitude'].astype(float)
JNorth_Peak_Migration_2001_12_df['longitude'] = JNorth_Peak_Migration_2001_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [98]:
geojson = df_to_geojson(JNorth_Peak_Migration_2001_12_df, cols)

In [99]:
file = open("JNorth_Peak_Migration_2001_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2002

In [100]:
JNorth_Peak_Migration_2002_df = JNorth_Peak_Migration_year_df[JNorth_Peak_Migration_year_df['year']==2002]
JNorth_Peak_Migration_2002_df.tail()

sighting_date                 species  latitude  longitude  number  year
464    2002-10-26  Monarch PEAK Migration    25.000   -100.000       1  2002
465    2002-10-26  Monarch PEAK Migration    25.420   -101.000       1  2002
466    2002-10-28  Monarch PEAK Migration    25.670   -100.300       1  2002
467    2002-10-28  Monarch PEAK Migration    26.800   -101.500       1  2002
468    2002-10-30  Monarch PEAK Migration    28.529    -96.679       1  2002

In [101]:
JNorth_Peak_Migration_2002_df.shape

(46, 6)

In [102]:
JNorth_Peak_Migration_2002_df = pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_df.drop(['year'], axis=1, inplace=True)

In [103]:
JNorth_Peak_Migration_2002_df.tail

<bound method NDFrame.tail of     sighting_date                 species  latitude  longitude  number
423    2002-08-19  Monarch PEAK Migration    44.906    -87.917       1
424    2002-08-27  Monarch PEAK Migration    46.089    -96.403       1
425    2002-08-28  Monarch PEAK Migration    44.504    -92.903       1
426    2002-08-28  Monarch PEAK Migration    47.368    -97.410       1
427    2002-09-08  Monarch PEAK Migration    40.127    -88.180       1
428    2002-09-08  Monarch PEAK Migration    44.953    -92.460       1
429    2002-09-10  Monarch PEAK Migration    41.080    -94.390       1
430    2002-09-10  Monarch PEAK Migration    41.420    -92.924       1
431    2002-09-10  Monarch PEAK Migration    45.724    -86.976       1
432    2002-09-11  Monarch PEAK Migration    40.755    -95.639       1
433    2002-09-11  Monarch PEAK Migration    44.278    -87.642       1
434    2002-09-14  Monarch PEAK Migration    41.420    -92.924       1
435    2002-09-17  Monarch PEAK Migration    39

In [104]:
JNorth_Peak_Migration_2002_df['month'] = pd.DatetimeIndex(JNorth_Peak_Migration_2002_df['sighting_date']).month

## January

In [105]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_1_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==1]
JNorth_Peak_Migration_2002_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [106]:
JNorth_Peak_Migration_2002_1_df.shape

(0, 6)

In [107]:
JNorth_Peak_Migration_2002_1_df = pd.DataFrame(JNorth_Peak_Migration_2002_1_df)
JNorth_Peak_Migration_2002_1_df.drop(['month'], axis=1, inplace=True)

In [108]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_1_df['latitude'] = JNorth_Peak_Migration_2002_1_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_1_df['longitude'] = JNorth_Peak_Migration_2002_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [109]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_1_df, cols)

In [110]:
file = open("JNorth_Peak_Migration_2002_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [111]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_2_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==2]
JNorth_Peak_Migration_2002_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [112]:
JNorth_Peak_Migration_2002_2_df.shape

(0, 6)

In [113]:
JNorth_Peak_Migration_2002_2_df = pd.DataFrame(JNorth_Peak_Migration_2002_2_df)
JNorth_Peak_Migration_2002_2_df.drop(['month'], axis=1, inplace=True)

In [114]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_2_df['latitude'] = JNorth_Peak_Migration_2002_2_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_2_df['longitude'] = JNorth_Peak_Migration_2002_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [115]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_2_df, cols)

In [116]:
file = open("JNorth_Peak_Migration_2002_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [117]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_3_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==3]
JNorth_Peak_Migration_2002_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [118]:
JNorth_Peak_Migration_2002_3_df.shape

(0, 6)

In [119]:
JNorth_Peak_Migration_2002_3_df = pd.DataFrame(JNorth_Peak_Migration_2002_3_df)
JNorth_Peak_Migration_2002_3_df.drop(['month'], axis=1, inplace=True)

In [120]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_3_df['latitude'] = JNorth_Peak_Migration_2002_3_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_3_df['longitude'] = JNorth_Peak_Migration_2002_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [121]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_3_df, cols)

In [122]:
file = open("JNorth_Peak_Migration_2002_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [123]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_4_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==4]
JNorth_Peak_Migration_2002_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [124]:
JNorth_Peak_Migration_2002_4_df.shape

(0, 6)

In [125]:
JNorth_Peak_Migration_2002_4_df = pd.DataFrame(JNorth_Peak_Migration_2002_4_df)
JNorth_Peak_Migration_2002_4_df.drop(['month'], axis=1, inplace=True)

In [126]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_4_df['latitude'] = JNorth_Peak_Migration_2002_4_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_4_df['longitude'] = JNorth_Peak_Migration_2002_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [127]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_4_df, cols)

In [128]:
file = open("JNorth_Peak_Migration_2002_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [129]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_5_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==5]
JNorth_Peak_Migration_2002_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [130]:
JNorth_Peak_Migration_2002_5_df.shape

(0, 6)

In [131]:
JNorth_Peak_Migration_2002_5_df = pd.DataFrame(JNorth_Peak_Migration_2002_5_df)
JNorth_Peak_Migration_2002_5_df.drop(['month'], axis=1, inplace=True)

In [132]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_5_df['latitude'] = JNorth_Peak_Migration_2002_5_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_5_df['longitude'] = JNorth_Peak_Migration_2002_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [133]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_5_df, cols)

In [134]:
file = open("JNorth_Peak_Migration_2002_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [135]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_6_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==6]
JNorth_Peak_Migration_2002_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [136]:
JNorth_Peak_Migration_2002_6_df.shape

(0, 6)

In [137]:
JNorth_Peak_Migration_2002_6_df = pd.DataFrame(JNorth_Peak_Migration_2002_6_df)
JNorth_Peak_Migration_2002_6_df.drop(['month'], axis=1, inplace=True)

In [138]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_6_df['latitude'] = JNorth_Peak_Migration_2002_6_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_6_df['longitude'] = JNorth_Peak_Migration_2002_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [139]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_6_df, cols)

In [140]:
file = open("JNorth_Peak_Migration_2002_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [141]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_7_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==7]
JNorth_Peak_Migration_2002_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [142]:
JNorth_Peak_Migration_2002_7_df.shape

(0, 6)

In [143]:
JNorth_Peak_Migration_2002_7_df = pd.DataFrame(JNorth_Peak_Migration_2002_7_df)
JNorth_Peak_Migration_2002_7_df.drop(['month'], axis=1, inplace=True)

In [144]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_7_df['latitude'] = JNorth_Peak_Migration_2002_7_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_7_df['longitude'] = JNorth_Peak_Migration_2002_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [145]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_7_df, cols)

In [146]:
file = open("JNorth_Peak_Migration_2002_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [147]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_8_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==8]
JNorth_Peak_Migration_2002_8_df.tail()

sighting_date                 species  latitude  longitude  number  month
423    2002-08-19  Monarch PEAK Migration    44.906    -87.917       1      8
424    2002-08-27  Monarch PEAK Migration    46.089    -96.403       1      8
425    2002-08-28  Monarch PEAK Migration    44.504    -92.903       1      8
426    2002-08-28  Monarch PEAK Migration    47.368    -97.410       1      8

In [148]:
JNorth_Peak_Migration_2002_8_df.shape

(4, 6)

In [149]:
JNorth_Peak_Migration_2002_8_df = pd.DataFrame(JNorth_Peak_Migration_2002_8_df)
JNorth_Peak_Migration_2002_8_df.drop(['month'], axis=1, inplace=True)

In [150]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_8_df['latitude'] = JNorth_Peak_Migration_2002_8_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_8_df['longitude'] = JNorth_Peak_Migration_2002_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [151]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_8_df, cols)

In [152]:
file = open("JNorth_Peak_Migration_2002_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [153]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_9_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==9]
JNorth_Peak_Migration_2002_9_df.tail()

sighting_date                 species  latitude  longitude  number  month
442    2002-09-23  Monarch PEAK Migration    43.090    -79.240       1      9
443    2002-09-26  Monarch PEAK Migration    40.525    -86.176       1      9
444    2002-09-29  Monarch PEAK Migration    42.030    -82.360       1      9
445    2002-09-29  Monarch PEAK Migration    43.702    -73.518       1      9
446    2002-09-30  Monarch PEAK Migration    43.510    -79.020       1      9

In [154]:
JNorth_Peak_Migration_2002_9_df.shape

(20, 6)

In [155]:
JNorth_Peak_Migration_2002_9_df = pd.DataFrame(JNorth_Peak_Migration_2002_9_df)
JNorth_Peak_Migration_2002_9_df.drop(['month'], axis=1, inplace=True)

In [156]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_9_df['latitude'] = JNorth_Peak_Migration_2002_9_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_9_df['longitude'] = JNorth_Peak_Migration_2002_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [157]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_9_df, cols)

In [158]:
file = open("JNorth_Peak_Migration_2002_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [159]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_10_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==10]
JNorth_Peak_Migration_2002_10_df.tail()

sighting_date                 species  latitude  longitude  number  month
464    2002-10-26  Monarch PEAK Migration    25.000   -100.000       1     10
465    2002-10-26  Monarch PEAK Migration    25.420   -101.000       1     10
466    2002-10-28  Monarch PEAK Migration    25.670   -100.300       1     10
467    2002-10-28  Monarch PEAK Migration    26.800   -101.500       1     10
468    2002-10-30  Monarch PEAK Migration    28.529    -96.679       1     10

In [160]:
JNorth_Peak_Migration_2002_10_df.shape

(22, 6)

In [161]:
JNorth_Peak_Migration_2002_10_df = pd.DataFrame(JNorth_Peak_Migration_2002_10_df)
JNorth_Peak_Migration_2002_10_df.drop(['month'], axis=1, inplace=True)

In [162]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_10_df['latitude'] = JNorth_Peak_Migration_2002_10_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_10_df['longitude'] = JNorth_Peak_Migration_2002_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [163]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_10_df, cols)

In [164]:
file = open("JNorth_Peak_Migration_2002_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [165]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_11_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==11]
JNorth_Peak_Migration_2002_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [166]:
JNorth_Peak_Migration_2002_11_df.shape

(0, 6)

In [167]:
JNorth_Peak_Migration_2002_11_df = pd.DataFrame(JNorth_Peak_Migration_2002_11_df)
JNorth_Peak_Migration_2002_11_df.drop(['month'], axis=1, inplace=True)

In [168]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_11_df['latitude'] = JNorth_Peak_Migration_2002_11_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_11_df['longitude'] = JNorth_Peak_Migration_2002_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [169]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_11_df, cols)

In [170]:
file = open("JNorth_Peak_Migration_2002_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [171]:
JNorth_Peak_Migration_2002_month_df =pd.DataFrame(JNorth_Peak_Migration_2002_df)
JNorth_Peak_Migration_2002_12_df = JNorth_Peak_Migration_2002_month_df[JNorth_Peak_Migration_2002_month_df['month']==12]
JNorth_Peak_Migration_2002_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [172]:
JNorth_Peak_Migration_2002_12_df.shape

(0, 6)

In [173]:
JNorth_Peak_Migration_2002_12_df = pd.DataFrame(JNorth_Peak_Migration_2002_12_df)
JNorth_Peak_Migration_2002_12_df.drop(['month'], axis=1, inplace=True)

In [174]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2002_12_df['latitude'] = JNorth_Peak_Migration_2002_12_df['latitude'].astype(float)
JNorth_Peak_Migration_2002_12_df['longitude'] = JNorth_Peak_Migration_2002_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [175]:
geojson = df_to_geojson(JNorth_Peak_Migration_2002_12_df, cols)

In [176]:
file = open("JNorth_Peak_Migration_2002_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2003

In [177]:
JNorth_Peak_Migration_2003_df = JNorth_Peak_Migration_year_df[JNorth_Peak_Migration_year_df['year']==2003]
JNorth_Peak_Migration_2003_df.tail()

sighting_date                 species  latitude  longitude  number  year
564    2003-10-29  Monarch PEAK Migration    35.887    -79.035       1  2003
565    2003-10-31  Monarch PEAK Migration    20.590   -100.400       1  2003
566    2003-11-01  Monarch PEAK Migration    20.590   -100.400       1  2003
567    2003-11-03  Monarch PEAK Migration    30.364    -90.040       1  2003
568    2003-11-13  Monarch PEAK Migration    35.887    -79.035       1  2003

In [178]:
JNorth_Peak_Migration_2003_df.shape

(100, 6)

In [179]:
JNorth_Peak_Migration_2003_df = pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_df.drop(['year'], axis=1, inplace=True)

In [180]:
JNorth_Peak_Migration_2003_df.tail

<bound method NDFrame.tail of     sighting_date                 species  latitude  longitude  number
469    2003-08-27  Monarch PEAK Migration    45.855    -84.864       1
470    2003-08-28  Monarch PEAK Migration    44.957    -93.540       1
471    2003-08-29  Monarch PEAK Migration    44.550    -78.550       1
472    2003-08-29  Monarch PEAK Migration    44.793    -94.198       1
473    2003-08-29  Monarch PEAK Migration    46.283    -96.098       1
..            ...                     ...       ...        ...     ...
564    2003-10-29  Monarch PEAK Migration    35.887    -79.035       1
565    2003-10-31  Monarch PEAK Migration    20.590   -100.400       1
566    2003-11-01  Monarch PEAK Migration    20.590   -100.400       1
567    2003-11-03  Monarch PEAK Migration    30.364    -90.040       1
568    2003-11-13  Monarch PEAK Migration    35.887    -79.035       1

[100 rows x 5 columns]>

In [181]:
JNorth_Peak_Migration_2003_df['month'] = pd.DatetimeIndex(JNorth_Peak_Migration_2003_df['sighting_date']).month

## January

In [182]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_1_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==1]
JNorth_Peak_Migration_2003_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [183]:
JNorth_Peak_Migration_2003_1_df.shape

(0, 6)

In [184]:
JNorth_Peak_Migration_2003_1_df = pd.DataFrame(JNorth_Peak_Migration_2003_1_df)
JNorth_Peak_Migration_2003_1_df.drop(['month'], axis=1, inplace=True)

In [185]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_1_df['latitude'] = JNorth_Peak_Migration_2003_1_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_1_df['longitude'] = JNorth_Peak_Migration_2003_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [186]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_1_df, cols)

In [187]:
file = open("JNorth_Peak_Migration_2003_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [188]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_2_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==2]
JNorth_Peak_Migration_2003_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [189]:
JNorth_Peak_Migration_2003_2_df.shape

(0, 6)

In [190]:
JNorth_Peak_Migration_2003_2_df = pd.DataFrame(JNorth_Peak_Migration_2003_2_df)
JNorth_Peak_Migration_2003_2_df.drop(['month'], axis=1, inplace=True)

In [191]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_2_df['latitude'] = JNorth_Peak_Migration_2003_2_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_2_df['longitude'] = JNorth_Peak_Migration_2003_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [192]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_2_df, cols)

In [193]:
file = open("JNorth_Peak_Migration_2003_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [194]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_3_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==3]
JNorth_Peak_Migration_2003_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [195]:
JNorth_Peak_Migration_2003_3_df.shape

(0, 6)

In [196]:
JNorth_Peak_Migration_2003_3_df = pd.DataFrame(JNorth_Peak_Migration_2003_3_df)
JNorth_Peak_Migration_2003_3_df.drop(['month'], axis=1, inplace=True)

In [197]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_3_df['latitude'] = JNorth_Peak_Migration_2003_3_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_3_df['longitude'] = JNorth_Peak_Migration_2003_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [198]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_3_df, cols)

In [199]:
file = open("JNorth_Peak_Migration_2003_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [200]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_4_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==4]
JNorth_Peak_Migration_2003_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [201]:
JNorth_Peak_Migration_2003_4_df.shape

(0, 6)

In [202]:
JNorth_Peak_Migration_2003_4_df =pd.DataFrame(JNorth_Peak_Migration_2003_4_df)
JNorth_Peak_Migration_2003_4_df.drop(['month'], axis=1, inplace=True)

In [203]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_4_df['latitude'] = JNorth_Peak_Migration_2003_4_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_4_df['longitude'] = JNorth_Peak_Migration_2003_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [204]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_4_df, cols)

In [205]:
file = open("JNorth_Peak_Migration_2003_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [206]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_5_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==5]
JNorth_Peak_Migration_2003_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [207]:
JNorth_Peak_Migration_2003_5_df.shape

(0, 6)

In [208]:
JNorth_Peak_Migration_2003_5_df =pd.DataFrame(JNorth_Peak_Migration_2003_5_df)
JNorth_Peak_Migration_2003_5_df.drop(['month'], axis=1, inplace=True)

In [209]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_5_df['latitude'] = JNorth_Peak_Migration_2003_5_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_5_df['longitude'] = JNorth_Peak_Migration_2003_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [210]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_5_df, cols)

In [211]:
file = open("JNorth_Peak_Migration_2003_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [212]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_6_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==6]
JNorth_Peak_Migration_2003_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [213]:
JNorth_Peak_Migration_2003_6_df.shape

(0, 6)

In [214]:
JNorth_Peak_Migration_2003_6_df = pd.DataFrame(JNorth_Peak_Migration_2003_6_df)
JNorth_Peak_Migration_2003_6_df.drop(['month'], axis=1, inplace=True)

In [215]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_6_df['latitude'] = JNorth_Peak_Migration_2003_6_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_6_df['longitude'] = JNorth_Peak_Migration_2003_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [216]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_6_df, cols)

In [217]:
file = open("JNorth_Peak_Migration_2003_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [218]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_7_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==7]
JNorth_Peak_Migration_2003_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [219]:
JNorth_Peak_Migration_2003_7_df.shape

(0, 6)

In [220]:
JNorth_Peak_Migration_2003_7_df = pd.DataFrame(JNorth_Peak_Migration_2003_7_df)
JNorth_Peak_Migration_2003_7_df.drop(['month'], axis=1, inplace=True)

In [221]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_7_df['latitude'] = JNorth_Peak_Migration_2003_7_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_7_df['longitude'] = JNorth_Peak_Migration_2003_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [222]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_7_df, cols)

In [223]:
file = open("JNorth_Peak_Migration_2003_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [224]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_8_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==8]
JNorth_Peak_Migration_2003_8_df.tail()

sighting_date                 species  latitude  longitude  number  month
470    2003-08-28  Monarch PEAK Migration    44.957    -93.540       1      8
471    2003-08-29  Monarch PEAK Migration    44.550    -78.550       1      8
472    2003-08-29  Monarch PEAK Migration    44.793    -94.198       1      8
473    2003-08-29  Monarch PEAK Migration    46.283    -96.098       1      8
474    2003-08-30  Monarch PEAK Migration    43.926    -95.229       1      8

In [225]:
JNorth_Peak_Migration_2003_8_df.shape

(6, 6)

In [226]:
JNorth_Peak_Migration_2003_8_df = pd.DataFrame(JNorth_Peak_Migration_2003_8_df)
JNorth_Peak_Migration_2003_8_df.drop(['month'], axis=1, inplace=True)

In [227]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_8_df['latitude'] = JNorth_Peak_Migration_2003_8_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_8_df['longitude'] = JNorth_Peak_Migration_2003_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [228]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_8_df, cols)

In [229]:
file = open("JNorth_Peak_Migration_2003_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [230]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_9_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==9]
JNorth_Peak_Migration_2003_9_df.tail()

sighting_date                 species  latitude  longitude  number  month
513    2003-09-28  Monarch PEAK Migration    36.642    -93.217       1      9
514    2003-09-28  Monarch PEAK Migration    37.452    -85.668       1      9
515    2003-09-29  Monarch PEAK Migration    35.290    -93.109       1      9
516    2003-09-29  Monarch PEAK Migration    39.643    -74.185       1      9
517    2003-09-29  Monarch PEAK Migration    40.584    -75.374       1      9

In [231]:
JNorth_Peak_Migration_2003_9_df.shape

(43, 6)

In [232]:
JNorth_Peak_Migration_2003_9_df = pd.DataFrame(JNorth_Peak_Migration_2003_9_df)
JNorth_Peak_Migration_2003_9_df.drop(['month'], axis=1, inplace=True)

In [233]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_9_df['latitude'] = JNorth_Peak_Migration_2003_9_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_9_df['longitude'] = JNorth_Peak_Migration_2003_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [234]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_9_df, cols)

In [235]:
file = open("JNorth_Peak_Migration_2003_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [236]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_10_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==10]
JNorth_Peak_Migration_2003_10_df.tail()

sighting_date                 species  latitude  longitude  number  month
561    2003-10-24  Monarch PEAK Migration    31.688    -89.111       1     10
562    2003-10-26  Monarch PEAK Migration    30.364    -90.040       1     10
563    2003-10-29  Monarch PEAK Migration    25.500   -100.200       1     10
564    2003-10-29  Monarch PEAK Migration    35.887    -79.035       1     10
565    2003-10-31  Monarch PEAK Migration    20.590   -100.400       1     10

In [237]:
JNorth_Peak_Migration_2003_10_df.shape

(48, 6)

In [238]:
JNorth_Peak_Migration_2003_10_df = pd.DataFrame(JNorth_Peak_Migration_2003_10_df)
JNorth_Peak_Migration_2003_10_df.drop(['month'], axis=1, inplace=True)

In [239]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_10_df['latitude'] = JNorth_Peak_Migration_2003_10_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_10_df['longitude'] = JNorth_Peak_Migration_2003_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [240]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_10_df, cols)

In [241]:
file = open("JNorth_Peak_Migration_2003_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [242]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_11_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==11]
JNorth_Peak_Migration_2003_11_df.tail()

sighting_date                 species  latitude  longitude  number  month
566    2003-11-01  Monarch PEAK Migration    20.590   -100.400       1     11
567    2003-11-03  Monarch PEAK Migration    30.364    -90.040       1     11
568    2003-11-13  Monarch PEAK Migration    35.887    -79.035       1     11

In [243]:
JNorth_Peak_Migration_2003_11_df.shape

(3, 6)

In [244]:
JNorth_Peak_Migration_2003_11_df = pd.DataFrame(JNorth_Peak_Migration_2003_11_df)
JNorth_Peak_Migration_2003_11_df.drop(['month'], axis=1, inplace=True)

In [245]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_11_df['latitude'] = JNorth_Peak_Migration_2003_11_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_11_df['longitude'] = JNorth_Peak_Migration_2003_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [246]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_11_df, cols)

In [247]:
file = open("JNorth_Peak_Migration_2003_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [248]:
JNorth_Peak_Migration_2003_month_df =pd.DataFrame(JNorth_Peak_Migration_2003_df)
JNorth_Peak_Migration_2003_12_df = JNorth_Peak_Migration_2003_month_df[JNorth_Peak_Migration_2003_month_df['month']==12]
JNorth_Peak_Migration_2003_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [249]:
JNorth_Peak_Migration_2003_12_df.shape

(0, 6)

In [250]:
JNorth_Peak_Migration_2003_12_df = pd.DataFrame(JNorth_Peak_Migration_2003_12_df)
JNorth_Peak_Migration_2003_12_df.drop(['month'], axis=1, inplace=True)

In [251]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2003_12_df['latitude'] = JNorth_Peak_Migration_2003_12_df['latitude'].astype(float)
JNorth_Peak_Migration_2003_12_df['longitude'] = JNorth_Peak_Migration_2003_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [252]:
geojson = df_to_geojson(JNorth_Peak_Migration_2003_12_df, cols)

In [253]:
file = open("JNorth_Peak_Migration_2003_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2004

In [254]:
JNorth_Peak_Migration_2004_df = JNorth_Peak_Migration_year_df[JNorth_Peak_Migration_year_df['year']==2004]
JNorth_Peak_Migration_2004_df.tail()

sighting_date                 species  latitude  longitude  number  year
620    2004-10-24  Monarch PEAK Migration    33.519    -86.808       1  2004
621    2004-10-25  Monarch PEAK Migration    34.915   -120.382       1  2004
622    2004-10-27  Monarch PEAK Migration    35.490   -120.911       1  2004
623    2004-10-29  Monarch PEAK Migration    32.323    -95.309       1  2004
624    2004-10-30  Monarch PEAK Migration    34.253    -85.124       1  2004

In [255]:
JNorth_Peak_Migration_2004_df.shape

(56, 6)

In [256]:
JNorth_Peak_Migration_2004_df = pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_df.drop(['year'], axis=1, inplace=True)

In [257]:
JNorth_Peak_Migration_2004_df.tail

<bound method NDFrame.tail of     sighting_date                 species  latitude  longitude  number
569    2004-08-01  Monarch PEAK Migration    19.620   -100.285       1
570    2004-09-02  Monarch PEAK Migration    44.669    -93.104       1
571    2004-09-05  Monarch PEAK Migration    42.180    -83.010       1
572    2004-09-18  Monarch PEAK Migration    40.099    -83.017       1
573    2004-09-21  Monarch PEAK Migration    43.420    -88.179       1
574    2004-09-22  Monarch PEAK Migration    41.555    -93.253       1
575    2004-09-23  Monarch PEAK Migration    42.132    -83.237       1
576    2004-09-24  Monarch PEAK Migration    40.480    -97.027       1
577    2004-09-25  Monarch PEAK Migration    40.735    -95.322       1
578    2004-09-25  Monarch PEAK Migration    41.468    -91.705       1
579    2004-09-25  Monarch PEAK Migration    45.186    -93.581       1
580    2004-09-26  Monarch PEAK Migration    42.418    -79.335       1
581    2004-09-26  Monarch PEAK Migration    43

In [258]:
JNorth_Peak_Migration_2004_df['month'] = pd.DatetimeIndex(JNorth_Peak_Migration_2004_df['sighting_date']).month

## January

In [259]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_1_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==1]
JNorth_Peak_Migration_2004_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [260]:
JNorth_Peak_Migration_2004_1_df.shape

(0, 6)

In [261]:
JNorth_Peak_Migration_2004_1_df = pd.DataFrame(JNorth_Peak_Migration_2004_1_df)
JNorth_Peak_Migration_2004_1_df.drop(['month'], axis=1, inplace=True)

In [262]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_1_df['latitude'] = JNorth_Peak_Migration_2004_1_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_1_df['longitude'] = JNorth_Peak_Migration_2004_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [263]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_1_df, cols)

In [264]:
file = open("JNorth_Peak_Migration_2004_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [265]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_2_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==2]
JNorth_Peak_Migration_2004_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [266]:
JNorth_Peak_Migration_2004_2_df.shape

(0, 6)

In [267]:
JNorth_Peak_Migration_2004_2_df = pd.DataFrame(JNorth_Peak_Migration_2004_2_df)
JNorth_Peak_Migration_2004_2_df.drop(['month'], axis=1, inplace=True)

In [268]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_2_df['latitude'] = JNorth_Peak_Migration_2004_2_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_2_df['longitude'] = JNorth_Peak_Migration_2004_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [269]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_2_df, cols)

In [270]:
file = open("JNorth_Peak_Migration_2004_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [271]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_3_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==3]
JNorth_Peak_Migration_2004_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [272]:
JNorth_Peak_Migration_2004_3_df.shape

(0, 6)

In [273]:
JNorth_Peak_Migration_2004_3_df = pd.DataFrame(JNorth_Peak_Migration_2004_3_df)
JNorth_Peak_Migration_2004_3_df.drop(['month'], axis=1, inplace=True)

In [274]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_3_df['latitude'] = JNorth_Peak_Migration_2004_3_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_3_df['longitude'] = JNorth_Peak_Migration_2004_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [275]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_3_df, cols)

In [276]:
file = open("JNorth_Peak_Migration_2004_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [277]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_4_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==4]
JNorth_Peak_Migration_2004_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [278]:
JNorth_Peak_Migration_2004_4_df.shape

(0, 6)

In [279]:
JNorth_Peak_Migration_2004_4_df = pd.DataFrame(JNorth_Peak_Migration_2004_4_df)
JNorth_Peak_Migration_2004_4_df.drop(['month'], axis=1, inplace=True)

In [280]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_4_df['latitude'] = JNorth_Peak_Migration_2004_4_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_4_df['longitude'] = JNorth_Peak_Migration_2004_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [281]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_4_df, cols)

In [282]:
file = open("JNorth_Peak_Migration_2004_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [283]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_5_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==5]
JNorth_Peak_Migration_2004_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [284]:
JNorth_Peak_Migration_2004_5_df.shape

(0, 6)

In [285]:
JNorth_Peak_Migration_2004_5_df = pd.DataFrame(JNorth_Peak_Migration_2004_5_df)
JNorth_Peak_Migration_2004_5_df.drop(['month'], axis=1, inplace=True)

In [286]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_5_df['latitude'] = JNorth_Peak_Migration_2004_5_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_5_df['longitude'] = JNorth_Peak_Migration_2004_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [287]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_5_df, cols)

In [288]:
file = open("JNorth_Peak_Migration_2004_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [289]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_6_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==6]
JNorth_Peak_Migration_2004_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [290]:
JNorth_Peak_Migration_2004_6_df.shape

(0, 6)

In [291]:
JNorth_Peak_Migration_2004_6_df = pd.DataFrame(JNorth_Peak_Migration_2004_6_df)
JNorth_Peak_Migration_2004_6_df.drop(['month'], axis=1, inplace=True)

In [292]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_6_df['latitude'] = JNorth_Peak_Migration_2004_6_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_6_df['longitude'] = JNorth_Peak_Migration_2004_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [293]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_6_df, cols)

In [294]:
file = open("JNorth_Peak_Migration_2004_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [295]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_7_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==7]
JNorth_Peak_Migration_2004_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [296]:
JNorth_Peak_Migration_2004_7_df.shape

(0, 6)

In [297]:
JNorth_Peak_Migration_2004_7_df = pd.DataFrame(JNorth_Peak_Migration_2004_7_df)
JNorth_Peak_Migration_2004_7_df.drop(['month'], axis=1, inplace=True)

In [298]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_7_df['latitude'] = JNorth_Peak_Migration_2004_7_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_7_df['longitude'] = JNorth_Peak_Migration_2004_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [299]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_7_df, cols)

In [300]:
file = open("JNorth_Peak_Migration_2004_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [301]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_8_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==8]
JNorth_Peak_Migration_2004_8_df.tail()

sighting_date                 species  latitude  longitude  number  month
569    2004-08-01  Monarch PEAK Migration     19.62   -100.285       1      8

In [302]:
JNorth_Peak_Migration_2004_8_df.shape

(1, 6)

In [303]:
JNorth_Peak_Migration_2004_8_df= pd.DataFrame(JNorth_Peak_Migration_2004_8_df)
JNorth_Peak_Migration_2004_8_df.drop(['month'], axis=1, inplace=True)

In [304]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_8_df['latitude'] = JNorth_Peak_Migration_2004_8_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_8_df['longitude'] = JNorth_Peak_Migration_2004_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [305]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_8_df, cols)

In [306]:
file = open("JNorth_Peak_Migration_2004_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [307]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_9_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==9]
JNorth_Peak_Migration_2004_9_df.tail()

sighting_date                 species  latitude  longitude  number  month
579    2004-09-25  Monarch PEAK Migration    45.186    -93.581       1      9
580    2004-09-26  Monarch PEAK Migration    42.418    -79.335       1      9
581    2004-09-26  Monarch PEAK Migration    43.270    -79.410       1      9
582    2004-09-27  Monarch PEAK Migration    33.197    -99.902       1      9
583    2004-09-27  Monarch PEAK Migration    39.645    -75.499       1      9

In [308]:
JNorth_Peak_Migration_2004_9_df.shape

(14, 6)

In [309]:
JNorth_Peak_Migration_2004_9_df = pd.DataFrame(JNorth_Peak_Migration_2004_9_df)
JNorth_Peak_Migration_2004_9_df.drop(['month'], axis=1, inplace=True)

In [310]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_9_df['latitude'] = JNorth_Peak_Migration_2004_9_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_9_df['longitude'] = JNorth_Peak_Migration_2004_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [311]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_9_df, cols)

In [312]:
file = open("JNorth_Peak_Migration_2004_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [313]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_10_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==10]
JNorth_Peak_Migration_2004_10_df.tail()

sighting_date                 species  latitude  longitude  number  month
620    2004-10-24  Monarch PEAK Migration    33.519    -86.808       1     10
621    2004-10-25  Monarch PEAK Migration    34.915   -120.382       1     10
622    2004-10-27  Monarch PEAK Migration    35.490   -120.911       1     10
623    2004-10-29  Monarch PEAK Migration    32.323    -95.309       1     10
624    2004-10-30  Monarch PEAK Migration    34.253    -85.124       1     10

In [314]:
JNorth_Peak_Migration_2004_10_df.shape

(41, 6)

In [315]:
JNorth_Peak_Migration_2004_10_df = pd.DataFrame(JNorth_Peak_Migration_2004_10_df)
JNorth_Peak_Migration_2004_10_df.drop(['month'], axis=1, inplace=True)

In [316]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_10_df['latitude'] = JNorth_Peak_Migration_2004_10_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_10_df['longitude'] = JNorth_Peak_Migration_2004_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [317]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_10_df, cols)

In [318]:
file = open("JNorth_Peak_Migration_2004_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [319]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_11_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==11]
JNorth_Peak_Migration_2004_11_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [320]:
JNorth_Peak_Migration_2004_11_df.shape

(0, 6)

In [321]:
JNorth_Peak_Migration_2004_11_df = pd.DataFrame(JNorth_Peak_Migration_2004_11_df)
JNorth_Peak_Migration_2004_11_df.drop(['month'], axis=1, inplace=True)

In [322]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_11_df['latitude'] = JNorth_Peak_Migration_2004_11_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_11_df['longitude'] = JNorth_Peak_Migration_2004_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [323]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_11_df, cols)

In [324]:
file = open("JNorth_Peak_Migration_2004_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [325]:
JNorth_Peak_Migration_2004_month_df =pd.DataFrame(JNorth_Peak_Migration_2004_df)
JNorth_Peak_Migration_2004_12_df = JNorth_Peak_Migration_2004_month_df[JNorth_Peak_Migration_2004_month_df['month']==12]
JNorth_Peak_Migration_2004_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [326]:
JNorth_Peak_Migration_2004_12_df.shape

(0, 6)

In [327]:
JNorth_Peak_Migration_2004_12_df = pd.DataFrame(JNorth_Peak_Migration_2004_12_df)
JNorth_Peak_Migration_2004_12_df.drop(['month'], axis=1, inplace=True)

In [328]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2004_12_df['latitude'] = JNorth_Peak_Migration_2004_12_df['latitude'].astype(float)
JNorth_Peak_Migration_2004_12_df['longitude'] = JNorth_Peak_Migration_2004_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [329]:
geojson = df_to_geojson(JNorth_Peak_Migration_2004_12_df, cols)

In [330]:
file = open("JNorth_Peak_Migration_2004_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2005

In [331]:
JNorth_Peak_Migration_2005_df = JNorth_Peak_Migration_df[JNorth_Peak_Migration_df['year']==2005]
JNorth_Peak_Migration_2005_df.tail()

sighting_date                 species  latitude  longitude  number  year
927    2005-10-31  Monarch PEAK Migration    25.170    -76.230       1  2005
928    2005-11-01  Monarch PEAK Migration    25.420   -101.000       1  2005
929    2005-11-03  Monarch PEAK Migration    19.620   -100.300       1  2005
930    2005-11-03  Monarch PEAK Migration    28.529    -96.679       1  2005
931    2005-11-07  Monarch PEAK Migration    20.310    -99.530       1  2005

In [332]:
JNorth_Peak_Migration_2005_df.shape

(307, 6)

In [333]:
JNorth_Peak_Migration_2005_df = pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_df.drop(['year'], axis=1, inplace=True)

In [334]:
JNorth_Peak_Migration_2005_df.tail

<bound method NDFrame.tail of     sighting_date                 species  latitude  longitude  number
625    2005-08-19  Monarch PEAK Migration    43.700    -79.420       1
626    2005-08-21  Monarch PEAK Migration    44.480    -93.155       1
627    2005-08-22  Monarch PEAK Migration    44.276    -94.188       1
628    2005-08-23  Monarch PEAK Migration    44.916    -97.170       1
629    2005-08-24  Monarch PEAK Migration    44.916    -97.170       1
..            ...                     ...       ...        ...     ...
927    2005-10-31  Monarch PEAK Migration    25.170    -76.230       1
928    2005-11-01  Monarch PEAK Migration    25.420   -101.000       1
929    2005-11-03  Monarch PEAK Migration    19.620   -100.300       1
930    2005-11-03  Monarch PEAK Migration    28.529    -96.679       1
931    2005-11-07  Monarch PEAK Migration    20.310    -99.530       1

[307 rows x 5 columns]>

In [335]:
JNorth_Peak_Migration_2005_df['month'] = pd.DatetimeIndex(JNorth_Peak_Migration_2005_df['sighting_date']).month

## January

In [336]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_1_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==1]
JNorth_Peak_Migration_2005_1_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [337]:
JNorth_Peak_Migration_2005_1_df.shape

(0, 6)

In [338]:
JNorth_Peak_Migration_2005_1_df = pd.DataFrame(JNorth_Peak_Migration_2005_1_df)
JNorth_Peak_Migration_2005_1_df.drop(['month'], axis=1, inplace=True)

In [339]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_1_df['latitude'] = JNorth_Peak_Migration_2005_1_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_1_df['longitude'] = JNorth_Peak_Migration_2005_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [340]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_1_df, cols)

In [341]:
file = open("JNorth_Peak_Migration_2005_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# February

In [342]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_2_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==2]
JNorth_Peak_Migration_2005_2_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [343]:
JNorth_Peak_Migration_2005_2_df.shape

(0, 6)

In [344]:
JNorth_Peak_Migration_2005_2_df = pd.DataFrame(JNorth_Peak_Migration_2005_2_df)
JNorth_Peak_Migration_2005_2_df.drop(['month'], axis=1, inplace=True)

In [345]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_2_df['latitude'] = JNorth_Peak_Migration_2005_2_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_2_df['longitude'] = JNorth_Peak_Migration_2005_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [346]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_2_df, cols)

In [347]:
file = open("JNorth_Peak_Migration_2005_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# March

In [348]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_3_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==3]
JNorth_Peak_Migration_2005_3_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [349]:
JNorth_Peak_Migration_2005_3_df.shape

(0, 6)

In [350]:
JNorth_Peak_Migration_2005_3_df = pd.DataFrame(JNorth_Peak_Migration_2005_3_df)
JNorth_Peak_Migration_2005_3_df.drop(['month'], axis=1, inplace=True)

In [351]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_3_df['latitude'] = JNorth_Peak_Migration_2005_3_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_3_df['longitude'] = JNorth_Peak_Migration_2005_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [352]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_3_df, cols)

In [353]:
file = open("JNorth_Peak_Migration_2005_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# April

In [354]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_4_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==4]
JNorth_Peak_Migration_2005_4_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [355]:
JNorth_Peak_Migration_2005_4_df.shape

(0, 6)

In [356]:
JNorth_Peak_Migration_2005_4_df= pd.DataFrame(JNorth_Peak_Migration_2005_4_df)
JNorth_Peak_Migration_2005_4_df.drop(['month'], axis=1, inplace=True)

In [357]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_4_df['latitude'] = JNorth_Peak_Migration_2005_4_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_4_df['longitude'] = JNorth_Peak_Migration_2005_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [358]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_4_df, cols)

In [359]:
file = open("JNorth_Peak_Migration_2005_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# May

In [360]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_5_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==5]
JNorth_Peak_Migration_2005_5_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [361]:
JNorth_Peak_Migration_2005_5_df.shape

(0, 6)

In [362]:
JNorth_Peak_Migration_2005_5_df = pd.DataFrame(JNorth_Peak_Migration_2005_5_df)
JNorth_Peak_Migration_2005_5_df.drop(['month'], axis=1, inplace=True)

In [363]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_5_df['latitude'] = JNorth_Peak_Migration_2005_5_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_5_df['longitude'] = JNorth_Peak_Migration_2005_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [364]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_5_df, cols)

In [365]:
file = open("JNorth_Peak_Migration_2005_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# June

In [366]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_6_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==6]
JNorth_Peak_Migration_2005_6_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [367]:
JNorth_Peak_Migration_2005_6_df.shape

(0, 6)

In [368]:
JNorth_Peak_Migration_2005_6_df = pd.DataFrame(JNorth_Peak_Migration_2005_6_df)
JNorth_Peak_Migration_2005_6_df.drop(['month'], axis=1, inplace=True)

In [369]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_6_df['latitude'] = JNorth_Peak_Migration_2005_6_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_6_df['longitude'] = JNorth_Peak_Migration_2005_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [370]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_6_df, cols)

In [371]:
file = open("JNorth_Peak_Migration_2005_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# July

In [372]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_7_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==7]
JNorth_Peak_Migration_2005_7_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [373]:
JNorth_Peak_Migration_2005_7_df.shape

(0, 6)

In [374]:
JNorth_Peak_Migration_2005_7_df = pd.DataFrame(JNorth_Peak_Migration_2005_7_df)
JNorth_Peak_Migration_2005_7_df.drop(['month'], axis=1, inplace=True)

In [375]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_7_df['latitude'] = JNorth_Peak_Migration_2005_7_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_7_df['longitude'] = JNorth_Peak_Migration_2005_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [376]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_7_df, cols)

In [377]:
file = open("JNorth_Peak_Migration_2005_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# August

In [378]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_8_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==8]
JNorth_Peak_Migration_2005_8_df.tail()

sighting_date                 species  latitude  longitude  number  month
642    2005-08-30  Monarch PEAK Migration    43.412    -96.671       1      8
643    2005-08-30  Monarch PEAK Migration    44.052    -98.679       1      8
644    2005-08-30  Monarch PEAK Migration    45.068    -93.013       1      8
645    2005-08-31  Monarch PEAK Migration    43.237    -86.254       1      8
646    2005-08-31  Monarch PEAK Migration    43.463    -83.710       1      8

In [379]:
JNorth_Peak_Migration_2005_8_df.shape

(22, 6)

In [380]:
JNorth_Peak_Migration_2005_8_df = pd.DataFrame(JNorth_Peak_Migration_2005_8_df)
JNorth_Peak_Migration_2005_8_df.drop(['month'], axis=1, inplace=True)

In [381]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_8_df['latitude'] = JNorth_Peak_Migration_2005_8_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_8_df['longitude'] = JNorth_Peak_Migration_2005_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [382]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_8_df, cols)

In [383]:
file = open("JNorth_Peak_Migration_2005_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# September

In [384]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_9_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==9]
JNorth_Peak_Migration_2005_9_df.tail()

sighting_date                 species  latitude  longitude  number  month
765    2005-09-29  Monarch PEAK Migration    37.538    -77.436       1      9
766    2005-09-29  Monarch PEAK Migration    39.201    -92.155       1      9
767    2005-09-30  Monarch PEAK Migration    34.494    -85.271       1      9
768    2005-09-30  Monarch PEAK Migration    38.940    -74.964       1      9
769    2005-09-30  Monarch PEAK Migration    38.968    -74.924       1      9

In [385]:
JNorth_Peak_Migration_2005_9_df.shape

(123, 6)

In [386]:
JNorth_Peak_Migration_2005_9_df = pd.DataFrame(JNorth_Peak_Migration_2005_9_df)
JNorth_Peak_Migration_2005_9_df.drop(['month'], axis=1, inplace=True)

In [387]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_9_df['latitude'] = JNorth_Peak_Migration_2005_9_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_9_df['longitude'] = JNorth_Peak_Migration_2005_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [388]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_9_df, cols)

In [389]:
file = open("JNorth_Peak_Migration_2005_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# October

In [390]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_10_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==10]
JNorth_Peak_Migration_2005_10_df.tail()

sighting_date                 species  latitude  longitude  number  month
923    2005-10-25  Monarch PEAK Migration    28.529    -96.679       1     10
924    2005-10-27  Monarch PEAK Migration    27.797    -97.399       1     10
925    2005-10-28  Monarch PEAK Migration    19.620   -100.285       1     10
926    2005-10-28  Monarch PEAK Migration    28.959    -82.626       1     10
927    2005-10-31  Monarch PEAK Migration    25.170    -76.230       1     10

In [391]:
JNorth_Peak_Migration_2005_10_df.shape

(158, 6)

In [392]:
JNorth_Peak_Migration_2005_10_df = pd.DataFrame(JNorth_Peak_Migration_2005_10_df)
JNorth_Peak_Migration_2005_10_df.drop(['month'], axis=1, inplace=True)

In [393]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_10_df['latitude'] = JNorth_Peak_Migration_2005_10_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_10_df['longitude'] = JNorth_Peak_Migration_2005_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [394]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_10_df, cols)

In [395]:
file = open("JNorth_Peak_Migration_2005_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# November

In [396]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_11_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==11]
JNorth_Peak_Migration_2005_11_df.tail()

sighting_date                 species  latitude  longitude  number  month
928    2005-11-01  Monarch PEAK Migration    25.420   -101.000       1     11
929    2005-11-03  Monarch PEAK Migration    19.620   -100.300       1     11
930    2005-11-03  Monarch PEAK Migration    28.529    -96.679       1     11
931    2005-11-07  Monarch PEAK Migration    20.310    -99.530       1     11

In [397]:
JNorth_Peak_Migration_2005_11_df.shape

(4, 6)

In [398]:
JNorth_Peak_Migration_2005_11_df = pd.DataFrame(JNorth_Peak_Migration_2005_11_df)
JNorth_Peak_Migration_2005_11_df.drop(['month'], axis=1, inplace=True)

In [399]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_11_df['latitude'] = JNorth_Peak_Migration_2005_11_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_11_df['longitude'] = JNorth_Peak_Migration_2005_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [400]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_11_df, cols)

In [401]:
file = open("JNorth_Peak_Migration_2005_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# December

In [402]:
JNorth_Peak_Migration_2005_month_df =pd.DataFrame(JNorth_Peak_Migration_2005_df)
JNorth_Peak_Migration_2005_12_df = JNorth_Peak_Migration_2005_month_df[JNorth_Peak_Migration_2005_month_df['month']==12]
JNorth_Peak_Migration_2005_12_df.tail()

Empty DataFrame
Columns: [sighting_date, species, latitude, longitude, number, month]
Index: []

In [403]:
JNorth_Peak_Migration_2005_12_df.shape

(0, 6)

In [404]:
JNorth_Peak_Migration_2005_12_df = pd.DataFrame(JNorth_Peak_Migration_2005_12_df)
JNorth_Peak_Migration_2005_12_df.drop(['month'], axis=1, inplace=True)

In [405]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Peak_Migration_2005_12_df['latitude'] = JNorth_Peak_Migration_2005_12_df['latitude'].astype(float)
JNorth_Peak_Migration_2005_12_df['longitude'] = JNorth_Peak_Migration_2005_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [406]:
geojson = df_to_geojson(JNorth_Peak_Migration_2005_12_df, cols)

In [407]:
file = open("JNorth_Peak_Migration_2005_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()